<a href="https://colab.research.google.com/github/herculesc/MachineLearning_Classification_FeaturesCNN/blob/main/SVM_2022_IC_INPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Criar novo dataset com os CSV (ok)
# Normalizar dados (ok)
# Aplicar modelos 
# Validação dos modelos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Bibliotecas**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
# y_true = y_teste
# y_pred = model.predict(x_teste)
dic_pevision = {}

def prevision(y_true, y_pred, final_treino):  
  precision = precision_score(y_true, y_pred , average="macro")
  f1 = f1_score(y_true, y_pred , average="macro")
  recall = recall_score(y_true, y_pred , average="macro")

  dic_pevision['f1-Score'] = "{:.4f}".format(f1)
  dic_pevision['Precision'] = "{:.4f}".format(precision)
  dic_pevision['Recall'] = "{:.4f}".format(recall)
  dic_pevision['tempo'] = "{:.4f}".format(final_treino)
  #print(f'F1: {f1:.4f} Precision:{precision:.4f} Recall:{recall:.4f} Tempo{final_treino:.4f}')

  return dic_pevision

In [ ]:
def Load_model(C_, Kernel, Degree):
  inicio_treino = init_time()
  model = make_pipeline(StandardScaler(),
                        SVC(C= C_,
                            kernel=Kernel,
                            degree=Degree,
                            gamma='auto',
                            decision_function_shape='ovr',
                            random_state=2))
  final_treino = time_ex(inicio_treino)
  
  model.fit(x_treino, y_treino)

  return {'Model':model, 'Tempo': final_treino}

In [ ]:
import sys
# Abrir local do arquivo

sys.path.append('/content/drive/MyDrive/TensorFlow_IC')

# Carrefar função do arquivo
from Time_execution import init_time
from Time_execution import time_ex
timeres = []

# **Carregar conjunto de dados**

In [ ]:
path = ['VGG_16', 'MobileNet', 'NASNetLarge', 'EfficientNetB0', 'EfficientNetB7', 'EfficientNetV2L' ]
train_path = []
test_path = []
for i in path:
  train = f'/content/drive/MyDrive/TensorFlow_IC/Dados/IC_Extraction_save/{i}/{i}_Train.csv'
  train_path.append(train)
  
  test = f'/content/drive/MyDrive/TensorFlow_IC/Dados/IC_Extraction_save/{i}/{i}_Test.csv'
  test_path.append(test)


In [ ]:
select_cnn = 5
selecao_train = train_path[select_cnn] 
selecao_test = test_path[select_cnn]

print(selecao_train)
print(selecao_test)

/content/drive/MyDrive/TensorFlow_IC/Dados/IC_Extraction_save/EfficientNetV2L/EfficientNetV2L_Train.csv
/content/drive/MyDrive/TensorFlow_IC/Dados/IC_Extraction_save/EfficientNetV2L/EfficientNetV2L_Test.csv


# **Carrega conjunto de dados**

In [ ]:
# Extração do conjunto de treinamento e alvo:
data = pd.read_csv(selecao_train)
alvo = pd.read_csv(selecao_test)

# Remover coluna vazia
data = data.drop(columns=['Unnamed: 0'])
alvo = alvo.drop(columns=['Unnamed: 0'])

In [ ]:
alvo.head()

,N1,N2,N3,N4,N5,N6,N7,N8,N9,N10,...,N992,N993,N994,N995,N996,N997,N998,N999,N1000,Classe
0,0.000371,0.000163,0.000153,0.000134,0.000150,0.000210,0.000147,0.000434,0.000474,0.000217,...,0.000126,0.000143,0.000361,0.000153,0.000468,0.000210,0.000071,0.000148,0.000384,0
1,0.000237,0.000139,0.000162,0.000145,0.000180,0.000351,0.000235,0.000380,0.000493,0.000384,...,0.000260,0.000236,0.000332,0.000205,0.000396,0.000338,0.000139,0.000356,0.000305,0
2,0.000220,0.000267,0.000180,0.000270,0.000365,0.000340,0.000304,0.000352,0.000346,0.000376,...,0.000373,0.000275,0.000446,0.000371,0.000819,0.000281,0.000263,0.000156,0.000547,0
3,0.000092,0.000071,0.000114,0.000090,0.000082,0.000153,0.000082,0.000091,0.000111,0.000119,...,0.000121,0.000100,0.000154,0.000162,0.000268,0.000114,0.000050,0.000071,0.000125,0
4,0.000186,0.000159,0.000227,0.000208,0.000184,0.000412,0.000230,0.000230,0.000300,0.000294,...,0.000280,0.000179,0.000351,0.000331,0.000502,0.000240,0.000121,0.000137,0.000317,0


In [ ]:
# Separação do conjunto alvo para validação:
esperado, alvo = alvo["Classe"], alvo.drop("Classe", axis=1)

In [ ]:
# Separação do conjunto de treinamento:
subteste = data.drop("Classe", axis=1)
valores = data["Classe"]

# **Preparação do conjunto para Logistic Regression**

In [ ]:
# Normatização dos dados:
x_treino, x_teste, y_treino, y_teste = train_test_split(subteste, valores, test_size=0.1)


In [ ]:
# Parametros
C_val	= [0.25, 0.500, 0.750, 1]
Kernel_val = ['linear', 'poly', 'rbf', 'sigmoid']
degree_val	= [3]

In [ ]:
# Cria dicionario com os dados
lista = []
cont = 0
for i in C_val:
  for y in Kernel_val:
    for z in degree_val:
      valor = [i, y, z]
      lista.append(valor)
      print(f'C: {i}  Kernel: "{y}" degree: {z}')

C: 0.25  Kernel: "linear" degree: 3
C: 0.25  Kernel: "poly" degree: 3
C: 0.25  Kernel: "rbf" degree: 3
C: 0.25  Kernel: "sigmoid" degree: 3
C: 0.5  Kernel: "linear" degree: 3
C: 0.5  Kernel: "poly" degree: 3
C: 0.5  Kernel: "rbf" degree: 3
C: 0.5  Kernel: "sigmoid" degree: 3
C: 0.75  Kernel: "linear" degree: 3
C: 0.75  Kernel: "poly" degree: 3
C: 0.75  Kernel: "rbf" degree: 3
C: 0.75  Kernel: "sigmoid" degree: 3
C: 1  Kernel: "linear" degree: 3
C: 1  Kernel: "poly" degree: 3
C: 1  Kernel: "rbf" degree: 3
C: 1  Kernel: "sigmoid" degree: 3


In [ ]:
# Treinamento do modelo
import pandas as pd
cnn_select = path[select_cnn]
print(f'CNN Selecionada: {cnn_select}')
output = pd.DataFrame()
for valores in lista:
  model_compile = Load_model(valores[0], valores[1], valores[2])
  modelo = model_compile['Model']
  tempo = model_compile['Tempo']
  
  y_true = y_teste
  y_pred = modelo.predict(x_teste)
  prev = prevision(y_true, y_pred, tempo)
  df_dictionary = pd.DataFrame([prev])
  output = pd.concat([output, df_dictionary], ignore_index=True)

  print(f'Penalty: {valores[0]}  C_val: {valores[1]} Max_val: {valores[2]} {prev}')

CNN Selecionada: EfficientNetV2L
Penalty: 0.25  C_val: linear Max_val: 3 {'f1-Score': '0.9889', 'Precision': '0.9890', 'Recall': '0.9888', 'tempo': '0.0002'}
Penalty: 0.25  C_val: poly Max_val: 3 {'f1-Score': '0.8753', 'Precision': '0.9167', 'Recall': '0.8648', 'tempo': '0.0002'}
Penalty: 0.25  C_val: rbf Max_val: 3 {'f1-Score': '0.9318', 'Precision': '0.9410', 'Recall': '0.9270', 'tempo': '0.0002'}
Penalty: 0.25  C_val: sigmoid Max_val: 3 {'f1-Score': '0.9665', 'Precision': '0.9656', 'Recall': '0.9681', 'tempo': '0.0002'}
Penalty: 0.5  C_val: linear Max_val: 3 {'f1-Score': '0.9929', 'Precision': '0.9923', 'Recall': '0.9938', 'tempo': '0.0001'}
Penalty: 0.5  C_val: poly Max_val: 3 {'f1-Score': '0.9110', 'Precision': '0.9371', 'Recall': '0.9015', 'tempo': '0.0002'}
Penalty: 0.5  C_val: rbf Max_val: 3 {'f1-Score': '0.9500', 'Precision': '0.9566', 'Recall': '0.9462', 'tempo': '0.0001'}
Penalty: 0.5  C_val: sigmoid Max_val: 3 {'f1-Score': '0.9614', 'Precision': '0.9597', 'Recall': '0.9659'

In [ ]:
output.head(50)

,f1-Score,Precision,Recall,tempo
0,0.9889,0.9890,0.9888,0.0002
1,0.8753,0.9167,0.8648,0.0002
2,0.9318,0.9410,0.9270,0.0002
3,0.9665,0.9656,0.9681,0.0002
4,0.9929,0.9923,0.9938,0.0001


Salvar_dataframe

In [ ]:
# writing to Excel
name_save = f'SVM_{cnn_select}_2022.xlsx'
datatoexcel = pd.ExcelWriter(name_save)

  
# write DataFrame to excel
output.to_excel(datatoexcel)
  
# save the excel
datatoexcel.save()

print(f'Arquivo: {name_save} salvo')
print('DataFrame is written to Excel File successfully.')

Arquivo: SVM_EfficientNetV2L_2022.xlsx salvo
DataFrame is written to Excel File successfully.
